In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import models
from tensorflow.keras import layers

import pandas as pd

2023-01-09 16:53:08.346117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 16:53:08.449637: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-09 16:53:08.449654: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-09 16:53:08.982395: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Na początku odczytano dane o samochodach z pliku CSV.

In [2]:
cars = pd.read_csv('car.data', encoding='utf-8')
cars

,buying,maint,doors,persons,lug_boot,safety,accept
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


Następnie zamieniono ciągi znaków na dyskretne wartości liczbowe:

In [3]:
cars['buying'].replace(['low', 'med', 'high', 'vhigh'], [0,1,2,3], inplace=True)
cars['maint'].replace(['low', 'med', 'high', 'vhigh'], [0,1,2,3], inplace=True)
cars['doors'].replace(['2', '3', '4', '5more'], [0,1,2,3], inplace=True)
cars['persons'].replace(['2', '4', 'more'], [0,1,2], inplace=True)
cars['lug_boot'].replace(['small', 'med', 'big'], [0,1,2], inplace=True)
cars['safety'].replace(['low', 'med', 'high'], [0,1,2], inplace=True)
cars['accept'].replace(['unacc', 'acc', 'good', 'vgood'], [0,1,2,3], inplace=True)

Następnie podzielono dane na zbiory parametrów i klas decyzyjnych:

In [4]:
from sklearn.model_selection import train_test_split

x = cars.drop("accept", axis=1)
y = cars["accept"]

Kolejnym krokiem było użycie drzewa decyzyjnego w celu sklasyfikowania danych, a także wygenerowanie metryk:

- MAE (Mean Absolute Error - średni błąd bezwzględny) i MSE (Mean Square Error - średni błąd kwadratowy), służących do pomiaru skuteczności regresji,
- True Positive Rate - określającej odsetek poprawnie sklasyfikowanych danych o klasie pozytywnej,
- True Negative Rate - określającej odsetek poprawnie sklasyfikowanych danych o klasie pozytywnej,
- False Positive Rate - określającej odsetek danych o klasie negatywnej błędnie sklasyfikowanych jako dane o klasie pozytywnej,
- False Negative Rate - określającej odsetek danych o klasie pozytywnej błędnie sklasyfikowanych jako dane o klasie negatywnej.

Dodatkowo obliczono średnią i odchylenie standardowe metryk MAE i MSE i uporządkowano kod, aby łatwiej można było ponownie wyliczyć metryki:

In [5]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error, classification_report

def display_errors(no_of_splits: int, dt_max_depth: int) -> None:
    predictions = []
    mean_absolute_errors = []
    mean_square_errors = []
    TPRs = []
    TNRs = []
    FPRs = []
    FNRs = []
    k_folds = KFold(n_splits = no_of_splits)
    DT_classifier = DecisionTreeClassifier(max_depth=dt_max_depth, criterion='entropy', random_state=42)

    for i, (train_index, test_index) in enumerate(k_folds.split(x)):
        X_train, X_test = x.iloc[train_index,:], x.iloc[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        DT_classifier.fit(X_train, y_train)
        prediction = DT_classifier.predict(X_test)
        cm = confusion_matrix(y_test, prediction)
        predictions.append(prediction)
        print("Confusion matrix for fold #" + str(i + 1))
        print(cm)
        print("Classification report for fold #" + str(i + 1))
        print(classification_report(y_test, prediction))
        # Wyświetlić tutaj metryki MAE, MSE, TP rate, TN rate, FP rate, FN rate, sumować do tablicy i pod koniec wyświetlić
        # średnią i odchylenie standardowe dla tych metryk
        mae = mean_absolute_error(y_test, prediction)
        mean_absolute_errors.append(mae)
        mse = mean_squared_error(y_test, prediction)
        mean_square_errors.append(mse)
        print("Mean absolute error for fold #" + str(i + 1))
        print(mae)
        print("Mean squared error for fold #" + str(i + 1))
        print(mse)
        FP = cm.sum(axis=0) - np.diag(cm)  
        FN = cm.sum(axis=1) - np.diag(cm)
        TP = np.diag(cm)
        TN = cm.sum() - (FP + FN + TP)

        print("TP rate for fold #" + str(i + 1) + " (unacc, acc, good, vgood)")
        TPR = TP/(TP+FN)
        TPRs.append(TPR)
        print(TPR)
        print("TN rate for fold #" + str(i + 1) + " (unacc, acc, good, vgood)")
        TNR = TN/(TN+FP) 
        TNRs.append(TNR)
        print(TNR)
        print("FP rate for fold #" + str(i + 1) + " (unacc, acc, good, vgood)")
        FPR = FP/(FP+TN)
        FPRs.append(FPR)
        print(FPR)
        print("FN rate for fold #" + str(i + 1) + " (unacc, acc, good, vgood)")
        FNR = FN/(TP+FN)
        FNRs.append(FNR)
        print(FNR)

        print("=======================================")
        
    print("Mean for MAE:")
    print(np.mean(np.asarray(mean_absolute_errors)))
    print("Stdev for MAE:")
    print(np.std(np.asarray(mean_absolute_errors)))
    print("Mean for MSE:")
    print(np.mean(np.asarray(mean_square_errors)))
    print("Stdev for MSE:")
    print(np.std(np.asarray(mean_square_errors)))
        
def train(test_size: float, no_of_splits: int, dt_max_depth: int) -> None:
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
    k_folds = KFold(n_splits = no_of_splits)
    DT_classifier = DecisionTreeClassifier(max_depth=dt_max_depth, criterion='entropy', random_state=42)
    DT_classifier.fit(X_train, y_train)
    y_pred = DT_classifier.predict(X_test)
    cf_matrix = confusion_matrix(y_test, y_pred)
    print(cf_matrix)
    print(classification_report(y_test, y_pred))


Następnie dokonano opisanych powyżej obliczeń, stosując różne wartości liczby podziałów i maksymalnej głębokości drzewa decyzyjnego, a następnie trenowano dalej sieć, stosując różne wielkości danych testowych:

In [6]:
display_errors(5, 5)

Confusion matrix for fold #1
[[242  64]
 [  0  40]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      0.79      0.88       306
           1       0.38      1.00      0.56        40

    accuracy                           0.82       346
   macro avg       0.69      0.90      0.72       346
weighted avg       0.93      0.82      0.85       346

Mean absolute error for fold #1
0.18497109826589594
Mean squared error for fold #1
0.18497109826589594
TP rate for fold #1 (unacc, acc, good, vgood)
[0.79084967 1.        ]
TN rate for fold #1 (unacc, acc, good, vgood)
[1.         0.79084967]
FP rate for fold #1 (unacc, acc, good, vgood)
[0.         0.20915033]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.20915033 0.        ]
Confusion matrix for fold #2
[[263   5]
 [ 44  34]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       0.86      0.98      0.91       268
         

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_12738/1447957

Potem wytrenowano model przy użyciu drzewa decyzyjnego:

In [7]:
train(0.25, 5, 5)

[[292  21   0   2]
 [ 18  63   0   2]
 [  0   7   0  13]
 [  0   5   0   9]]
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       315
           1       0.66      0.76      0.70        83
           2       0.00      0.00      0.00        20
           3       0.35      0.64      0.45        14

    accuracy                           0.84       432
   macro avg       0.49      0.58      0.52       432
weighted avg       0.82      0.84      0.83       432



/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Wartość metryki accuracy wynosi 0.85 co oznacza, że poprawność przewidywania wynosi 85%.

Potem powtórzono eksperyment poprzez zmianę parametru max depth, który wyniósł 15.

In [8]:
display_errors(5, 15)

Confusion matrix for fold #1
[[270  36]
 [  0  40]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      0.88      0.94       306
           1       0.53      1.00      0.69        40

    accuracy                           0.90       346
   macro avg       0.76      0.94      0.81       346
weighted avg       0.95      0.90      0.91       346

Mean absolute error for fold #1
0.10404624277456648
Mean squared error for fold #1
0.10404624277456648
TP rate for fold #1 (unacc, acc, good, vgood)
[0.88235294 1.        ]
TN rate for fold #1 (unacc, acc, good, vgood)
[1.         0.88235294]
FP rate for fold #1 (unacc, acc, good, vgood)
[0.         0.11764706]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.11764706 0.        ]
Confusion matrix for fold #2
[[268   0]
 [ 36  42]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       268
         

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Wartości MAE i MSE są nieco mniejsze niż w poprzednim przykładzie. Potem ponownie przeprowadzono trenowanie modelu i tym razem accuracy wynosiła 98% co jest wynikiem lepszym niż poprzednio.

In [9]:
train(0.25, 5, 15)

[[298   1   2   0]
 [  1 100   1   1]
 [  1   2  11   0]
 [  0   0   0  14]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       301
           1       0.97      0.97      0.97       103
           2       0.79      0.79      0.79        14
           3       0.93      1.00      0.97        14

    accuracy                           0.98       432
   macro avg       0.92      0.94      0.93       432
weighted avg       0.98      0.98      0.98       432



Następnie przeprowadzono eksperyment ponownie, lecz tym razem przy max depth wynoszącym 30. Wyniki błędów okazały się być takie same jak w przypadku maxdepth równym 15.

In [10]:
display_errors(5, 30)

Confusion matrix for fold #1
[[270  36]
 [  0  40]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      0.88      0.94       306
           1       0.53      1.00      0.69        40

    accuracy                           0.90       346
   macro avg       0.76      0.94      0.81       346
weighted avg       0.95      0.90      0.91       346

Mean absolute error for fold #1
0.10404624277456648
Mean squared error for fold #1
0.10404624277456648
TP rate for fold #1 (unacc, acc, good, vgood)
[0.88235294 1.        ]
TN rate for fold #1 (unacc, acc, good, vgood)
[1.         0.88235294]
FP rate for fold #1 (unacc, acc, good, vgood)
[0.         0.11764706]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.11764706 0.        ]
Confusion matrix for fold #2
[[268   0]
 [ 36  42]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       268
         

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
train(0.25, 5, 30)

[[291   3   0   0]
 [  6  97   0   0]
 [  0   1  19   0]
 [  0   0   2  13]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       294
           1       0.96      0.94      0.95       103
           2       0.90      0.95      0.93        20
           3       1.00      0.87      0.93        15

    accuracy                           0.97       432
   macro avg       0.96      0.94      0.95       432
weighted avg       0.97      0.97      0.97       432



Jak widać powyżej, wyniki uczenia modelu były bardzo zbliżone do wartości, gdy maxdepth wynosi 15.

Następnie wyświetlono średnie błędów dla 15 sąsiadów użytych w walidacji krzyżowej.

In [12]:
display_errors(15, 5)

Confusion matrix for fold #1
[[116]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       116

    accuracy                           1.00       116
   macro avg       1.00      1.00      1.00       116
weighted avg       1.00      1.00      1.00       116

Mean absolute error for fold #1
0.0
Mean squared error for fold #1
0.0
TP rate for fold #1 (unacc, acc, good, vgood)
[1.]
TN rate for fold #1 (unacc, acc, good, vgood)
[nan]
FP rate for fold #1 (unacc, acc, good, vgood)
[nan]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.]
Confusion matrix for fold #2
[[81 33]
 [ 1  1]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       0.99      0.71      0.83       114
           1       0.03      0.50      0.06         2

    accuracy                           0.71       116
   macro avg       0.51      0.61      0.44       116
weighted avg       0.97  

/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetric

Confusion matrix for fold #8
[[69  8]
 [ 4 34]]
Classification report for fold #8
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        77
           1       0.81      0.89      0.85        38

    accuracy                           0.90       115
   macro avg       0.88      0.90      0.89       115
weighted avg       0.90      0.90      0.90       115

Mean absolute error for fold #8
0.10434782608695652
Mean squared error for fold #8
0.10434782608695652
TP rate for fold #8 (unacc, acc, good, vgood)
[0.8961039  0.89473684]
TN rate for fold #8 (unacc, acc, good, vgood)
[0.89473684 0.8961039 ]
FP rate for fold #8 (unacc, acc, good, vgood)
[0.10526316 0.1038961 ]
FN rate for fold #8 (unacc, acc, good, vgood)
[0.1038961  0.10526316]
Confusion matrix for fold #9
[[65 13]
 [ 0 37]]
Classification report for fold #9
              precision    recall  f1-score   support

           0       1.00      0.83      0.91        78
           1     

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Confusion matrix for fold #14
[[62  2  0  0]
 [ 0 16  0  0]
 [ 0 20  0  0]
 [ 0 15  0  0]]
Classification report for fold #14
              precision    recall  f1-score   support

           0       1.00      0.97      0.98        64
           1       0.30      1.00      0.46        16
           2       0.00      0.00      0.00        20
           3       0.00      0.00      0.00        15

    accuracy                           0.68       115
   macro avg       0.33      0.49      0.36       115
weighted avg       0.60      0.68      0.61       115

Mean absolute error for fold #14
0.45217391304347826
Mean squared error for fold #14
0.7130434782608696
TP rate for fold #14 (unacc, acc, good, vgood)
[0.96875 1.      0.      0.     ]
TN rate for fold #14 (unacc, acc, good, vgood)
[1.         0.62626263 1.         1.        ]
FP rate for fold #14 (unacc, acc, good, vgood)
[0.         0.37373737 0.         0.        ]
FN rate for fold #14 (unacc, acc, good, vgood)
[0.03125 0.      1.  

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Wyniki okazały się niewiele lepsze niż w przypadku wyników z poprzedniego ćwiczenia.

In [13]:
train(0.25, 15, 5)

[[286  31   0   0]
 [  5  72   0   2]
 [  0   8   0   9]
 [  0   3   0  16]]
              precision    recall  f1-score   support

           0       0.98      0.90      0.94       317
           1       0.63      0.91      0.75        79
           2       0.00      0.00      0.00        17
           3       0.59      0.84      0.70        19

    accuracy                           0.87       432
   macro avg       0.55      0.66      0.60       432
weighted avg       0.86      0.87      0.86       432



/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tak samo wyniki uczenia okazały się być nieznacznie lepsze od wyników z poprzedniego ćwiczenia. Wynik accuracy wynosi 87%.

Następnie powtórzono eksperyment przy użyciu 30 sąsiadów i maxdepth równego 5. Wartości MAE i MSE są niższe niż w poprzednim przypadku, lecz nie jest to duża zmiana.

In [14]:
display_errors(30, 5)

Confusion matrix for fold #1
[[58]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #1
0.0
Mean squared error for fold #1
0.0
TP rate for fold #1 (unacc, acc, good, vgood)
[1.]
TN rate for fold #1 (unacc, acc, good, vgood)
[nan]
FP rate for fold #1 (unacc, acc, good, vgood)
[nan]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.]
Confusion matrix for fold #2
[[58]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #2
0.0
M

/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` par

              precision    recall  f1-score   support

           0       1.00      0.79      0.89        39
           1       0.70      1.00      0.83        19

    accuracy                           0.86        58
   macro avg       0.85      0.90      0.86        58
weighted avg       0.90      0.86      0.87        58

Mean absolute error for fold #5
0.13793103448275862
Mean squared error for fold #5
0.13793103448275862
TP rate for fold #5 (unacc, acc, good, vgood)
[0.79487179 1.        ]
TN rate for fold #5 (unacc, acc, good, vgood)
[1.         0.79487179]
FP rate for fold #5 (unacc, acc, good, vgood)
[0.         0.20512821]
FN rate for fold #5 (unacc, acc, good, vgood)
[0.20512821 0.        ]
Confusion matrix for fold #6
[[31  7]
 [ 0 20]]
Classification report for fold #6
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        38
           1       0.74      1.00      0.85        20

    accuracy                           0.88 

/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetric

Confusion matrix for fold #12
[[32  8]
 [ 6 12]]
Classification report for fold #12
              precision    recall  f1-score   support

           0       0.84      0.80      0.82        40
           1       0.60      0.67      0.63        18

    accuracy                           0.76        58
   macro avg       0.72      0.73      0.73        58
weighted avg       0.77      0.76      0.76        58

Mean absolute error for fold #12
0.2413793103448276
Mean squared error for fold #12
0.2413793103448276
TP rate for fold #12 (unacc, acc, good, vgood)
[0.8        0.66666667]
TN rate for fold #12 (unacc, acc, good, vgood)
[0.66666667 0.8       ]
FP rate for fold #12 (unacc, acc, good, vgood)
[0.33333333 0.2       ]
FN rate for fold #12 (unacc, acc, good, vgood)
[0.2        0.33333333]
Confusion matrix for fold #13
[[36  0]
 [ 4 18]]
Classification report for fold #13
              precision    recall  f1-score   support

           0       0.90      1.00      0.95        36
         

...

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Confusion matrix for fold #28
[[31  0  0  0]
 [ 0  5  0  0]
 [ 0  8  0  5]
 [ 0  0  0  8]]
Classification report for fold #28
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.38      1.00      0.56         5
           2       0.00      0.00      0.00        13
           3       0.62      1.00      0.76         8

    accuracy                           0.77        57
   macro avg       0.50      0.75      0.58        57
weighted avg       0.66      0.77      0.70        57

Mean absolute error for fold #28
0.22807017543859648
Mean squared error for fold #28
0.22807017543859648
TP rate for fold #28 (unacc, acc, good, vgood)
[1. 1. 0. 1.]
TN rate for fold #28 (unacc, acc, good, vgood)
[1.         0.84615385 1.         0.89795918]
FP rate for fold #28 (unacc, acc, good, vgood)
[0.         0.15384615 0.         0.10204082]
FN rate for fold #28 (unacc, acc, good, vgood)
[0. 0. 1. 0.]
Confusion matrix for fold 

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hom

Confusion matrix for fold #30
[[31  0  0  0]
 [ 0  4  0  0]
 [ 0  9  0  4]
 [ 0  0  0  9]]
Classification report for fold #30
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.31      1.00      0.47         4
           2       0.00      0.00      0.00        13
           3       0.69      1.00      0.82         9

    accuracy                           0.77        57
   macro avg       0.50      0.75      0.57        57
weighted avg       0.67      0.77      0.71        57

Mean absolute error for fold #30
0.22807017543859648
Mean squared error for fold #30
0.22807017543859648
TP rate for fold #30 (unacc, acc, good, vgood)
[1. 1. 0. 1.]
TN rate for fold #30 (unacc, acc, good, vgood)
[1.         0.83018868 1.         0.91666667]
FP rate for fold #30 (unacc, acc, good, vgood)
[0.         0.16981132 0.         0.08333333]
FN rate for fold #30 (unacc, acc, good, vgood)
[0. 0. 1. 0.]
Mean for MAE:
0.1682395644

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
train(0.25, 30, 5)

[[286  29   0   2]
 [  7  74   0   3]
 [  0   8   0   6]
 [  0   4   0  13]]
              precision    recall  f1-score   support

           0       0.98      0.90      0.94       317
           1       0.64      0.88      0.74        84
           2       0.00      0.00      0.00        14
           3       0.54      0.76      0.63        17

    accuracy                           0.86       432
   macro avg       0.54      0.64      0.58       432
weighted avg       0.86      0.86      0.86       432



/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Wyniki trenowania również nie wykazały większych zmian w trafności przewidywania. Wynik accuracy wyniósł 86%.

In [16]:
display_errors(15, 15)

Confusion matrix for fold #1
[[116]]
Classification report for fold #1


/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       116

    accuracy                           1.00       116
   macro avg       1.00      1.00      1.00       116
weighted avg       1.00      1.00      1.00       116

Mean absolute error for fold #1
0.0
Mean squared error for fold #1
0.0
TP rate for fold #1 (unacc, acc, good, vgood)
[1.]
TN rate for fold #1 (unacc, acc, good, vgood)
[nan]
FP rate for fold #1 (unacc, acc, good, vgood)
[nan]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.]
Confusion matrix for fold #2
[[78 36]
 [ 0  2]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       1.00      0.68      0.81       114
           1       0.05      1.00      0.10         2

    accuracy                           0.69       116
   macro avg       0.53      0.84      0.46       116
weighted avg       0.98      0.69      0.80       116

Mean absolute error for fold #2
0.3103448

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_12738/1447957

Zmiana wartości k-fold na 15 oraz maxdepth na 15 dała jednak znaczną poprawę, wartości MAE i MSE są znacznie niższe, niż w przypadku użycia maxdepth równego 5. Wynik accuracy również uległ poprawie, wynosi teraz 98%.

In [17]:
train(0.25, 15, 15)

[[303   2   0   0]
 [  3  91   0   1]
 [  0   2  16   0]
 [  0   0   2  12]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       305
           1       0.96      0.96      0.96        95
           2       0.89      0.89      0.89        18
           3       0.92      0.86      0.89        14

    accuracy                           0.98       432
   macro avg       0.94      0.92      0.93       432
weighted avg       0.98      0.98      0.98       432



Następnie zmieniono maxdepth na 30. Wyniki uczenia oraz wartości MAE i MSE nie uległy zmianie.

In [18]:
display_errors(15, 30)

/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)


Confusion matrix for fold #1
[[116]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       116

    accuracy                           1.00       116
   macro avg       1.00      1.00      1.00       116
weighted avg       1.00      1.00      1.00       116

Mean absolute error for fold #1
0.0
Mean squared error for fold #1
0.0
TP rate for fold #1 (unacc, acc, good, vgood)
[1.]
TN rate for fold #1 (unacc, acc, good, vgood)
[nan]
FP rate for fold #1 (unacc, acc, good, vgood)
[nan]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.]
Confusion matrix for fold #2
[[78 36]
 [ 0  2]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       1.00      0.68      0.81       114
           1       0.05      1.00      0.10         2

    accuracy                           0.69       116
   macro avg       0.53      0.84      0.46       116
weighted avg       0.98  

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_12738/1447957

Confusion matrix for fold #11
[[63  0  1  0]
 [ 0 20  0  0]
 [ 0  8  9  0]
 [ 0  0  0 14]]
Classification report for fold #11
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        64
           1       0.71      1.00      0.83        20
           2       0.90      0.53      0.67        17
           3       1.00      1.00      1.00        14

    accuracy                           0.92       115
   macro avg       0.90      0.88      0.87       115
weighted avg       0.94      0.92      0.92       115

Mean absolute error for fold #11
0.08695652173913043
Mean squared error for fold #11
0.10434782608695652
TP rate for fold #11 (unacc, acc, good, vgood)
[0.984375   1.         0.52941176 1.        ]
TN rate for fold #11 (unacc, acc, good, vgood)
[1.         0.91578947 0.98979592 1.        ]
FP rate for fold #11 (unacc, acc, good, vgood)
[0.         0.08421053 0.01020408 0.        ]
FN rate for fold #11 (unacc, acc, good, vgood)
[0.01562

In [19]:
train(0.25, 15, 30)

[[310   2   0   0]
 [  3  82   2   0]
 [  0   1  15   0]
 [  0   1   0  16]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       312
           1       0.95      0.94      0.95        87
           2       0.88      0.94      0.91        16
           3       1.00      0.94      0.97        17

    accuracy                           0.98       432
   macro avg       0.96      0.95      0.95       432
weighted avg       0.98      0.98      0.98       432



Następnie zmieniono parametr k-foldów na 30 oraz wartość maxdepth na 15. Błędy MAE i MSE okazały się być znacznie niższe niż w poprzednich przykładach.

In [20]:
display_errors(30, 15)

Confusion matrix for fold #1
[[58]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #1
0.0
Mean squared error for fold #1
0.0
TP rate for fold #1 (unacc, acc, good, vgood)
[1.]
TN rate for fold #1 (unacc, acc, good, vgood)
[nan]
FP rate for fold #1 (unacc, acc, good, vgood)
[nan]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.]


/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)


Confusion matrix for fold #2
[[58]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #2
0.0
Mean squared error for fold #2
0.0
TP rate for fold #2 (unacc, acc, good, vgood)
[1.]
TN rate for fold #2 (unacc, acc, good, vgood)
[nan]
FP rate for fold #2 (unacc, acc, good, vgood)
[nan]
FN rate for fold #2 (unacc, acc, good, vgood)
[0.]
Confusion matrix for fold #3
[[58]]
Classification report for fold #3
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #3
0.0
M

/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)


Confusion matrix for fold #12
[[40  0]
 [ 0 18]]
Classification report for fold #12
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        18

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #12
0.0
Mean squared error for fold #12
0.0
TP rate for fold #12 (unacc, acc, good, vgood)
[1. 1.]
TN rate for fold #12 (unacc, acc, good, vgood)
[1. 1.]
FP rate for fold #12 (unacc, acc, good, vgood)
[0. 0.]
FN rate for fold #12 (unacc, acc, good, vgood)
[0. 0.]
Confusion matrix for fold #13
[[36  0]
 [ 0 22]]
Classification report for fold #13
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        22

    accuracy                           1.00     

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_12738/1447957

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      0.95      0.97        19
           2       0.00      0.00      0.00         0
           3       1.00      1.00      1.00         7

    accuracy                           0.98        57
   macro avg       0.75      0.74      0.74        57
weighted avg       1.00      0.98      0.99        57

Mean absolute error for fold #20
0.017543859649122806
Mean squared error for fold #20
0.017543859649122806
TP rate for fold #20 (unacc, acc, good, vgood)
[1.         0.94736842        nan 1.        ]
TN rate for fold #20 (unacc, acc, good, vgood)
[1.         1.         0.98245614 1.        ]
FP rate for fold #20 (unacc, acc, good, vgood)
[0.         0.         0.01754386 0.        ]
FN rate for fold #20 (unacc, acc, good, vgood)
[0.         0.05263158        nan 0.        ]
Confusion matrix for fold #21
[[33  0  0  0]
 [ 0 12  0  0]
 [ 0  1  3  1]
 [ 0  0  0 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        12
           3       1.00      1.00      1.00         6

    accuracy                           1.00        57
   macro avg       1.00      1.00      1.00        57
weighted avg       1.00      1.00      1.00        57

Mean absolute error for fold #29
0.0
Mean squared error for fold #29
0.0
TP rate for fold #29 (unacc, acc, good, vgood)
[1. 1. 1. 1.]
TN rate for fold #29 (unacc, acc, good, vgood)
[1. 1. 1. 1.]
FP rate for fold #29 (unacc, acc, good, vgood)
[0. 0. 0. 0.]
FN rate for fold #29 (unacc, acc, good, vgood)
[0. 0. 0. 0.]
Confusion matrix for fold #30
[[31  0  0  0]
 [ 0  4  0  0]
 [ 0  0 13  0]
 [ 0  0  0  9]]
Classification report for fold #30
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1 

In [21]:
train(0.25, 30, 15)

[[303   3   0   0]
 [  1  89   2   0]
 [  1   4  16   0]
 [  0   0   0  13]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       306
           1       0.93      0.97      0.95        92
           2       0.89      0.76      0.82        21
           3       1.00      1.00      1.00        13

    accuracy                           0.97       432
   macro avg       0.95      0.93      0.94       432
weighted avg       0.97      0.97      0.97       432



Potem zmieniono oba parametry na 30. Wyniki uczenia modelu lub błędów MAE i MSE nie uległy znaczącej poprawie.

In [22]:
display_errors(30, 30)

/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)
/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)


Confusion matrix for fold #1
[[58]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #1
0.0
Mean squared error for fold #1
0.0
TP rate for fold #1 (unacc, acc, good, vgood)
[1.]
TN rate for fold #1 (unacc, acc, good, vgood)
[nan]
FP rate for fold #1 (unacc, acc, good, vgood)
[nan]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.]
Confusion matrix for fold #2
[[58]]
Classification report for fold #2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #2
0.0
M

Confusion matrix for fold #13
[[36  0]
 [ 0 22]]
Classification report for fold #13
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        22

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #13
0.0
Mean squared error for fold #13
0.0
TP rate for fold #13 (unacc, acc, good, vgood)
[1. 1.]
TN rate for fold #13 (unacc, acc, good, vgood)
[1. 1.]
FP rate for fold #13 (unacc, acc, good, vgood)
[0. 0.]
FN rate for fold #13 (unacc, acc, good, vgood)
[0. 0.]
Confusion matrix for fold #14
[[40  0]
 [ 0 18]]
Classification report for fold #14


/tmp/ipykernel_12738/1447957055.py:47: RuntimeWarning: invalid value encountered in divide
  TNR = TN/(TN+FP)
/tmp/ipykernel_12738/1447957055.py:51: RuntimeWarning: invalid value encountered in divide
  FPR = FP/(FP+TN)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        18

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58

Mean absolute error for fold #14
0.0
Mean squared error for fold #14
0.0
TP rate for fold #14 (unacc, acc, good, vgood)
[1. 1.]
TN rate for fold #14 (unacc, acc, good, vgood)
[1. 1.]
FP rate for fold #14 (unacc, acc, good, vgood)
[0. 0.]
FN rate for fold #14 (unacc, acc, good, vgood)
[0. 0.]
Confusion matrix for fold #15
[[38  0]
 [ 0 20]]
Classification report for fold #15
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        20

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00 

Confusion matrix for fold #25
[[33  2  0]
 [ 2 17  1]
 [ 0  0  2]]
Classification report for fold #25
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        35
           1       0.89      0.85      0.87        20
           3       0.67      1.00      0.80         2

    accuracy                           0.91        57
   macro avg       0.83      0.93      0.87        57
weighted avg       0.92      0.91      0.91        57

Mean absolute error for fold #25
0.10526315789473684
Mean squared error for fold #25
0.14035087719298245
TP rate for fold #25 (unacc, acc, good, vgood)
[0.94285714 0.85       1.        ]
TN rate for fold #25 (unacc, acc, good, vgood)
[0.90909091 0.94594595 0.98181818]
FP rate for fold #25 (unacc, acc, good, vgood)
[0.09090909 0.05405405 0.01818182]
FN rate for fold #25 (unacc, acc, good, vgood)
[0.05714286 0.15       0.        ]
Confusion matrix for fold #26
[[31  0  0]
 [ 0 18  0]
 [ 0  1  7]]
Classification re

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_12738/1447957

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        12
           3       1.00      1.00      1.00         6

    accuracy                           1.00        57
   macro avg       1.00      1.00      1.00        57
weighted avg       1.00      1.00      1.00        57

Mean absolute error for fold #29
0.0
Mean squared error for fold #29
0.0
TP rate for fold #29 (unacc, acc, good, vgood)
[1. 1. 1. 1.]
TN rate for fold #29 (unacc, acc, good, vgood)
[1. 1. 1. 1.]
FP rate for fold #29 (unacc, acc, good, vgood)
[0. 0. 0. 0.]
FN rate for fold #29 (unacc, acc, good, vgood)
[0. 0. 0. 0.]
Confusion matrix for fold #30
[[31  0  0  0]
 [ 0  4  0  0]
 [ 0  0 13  0]
 [ 0  0  0  9]]
Classification report for fold #30
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1 

In [23]:
train(0.25, 30, 30)

[[310   2   0   0]
 [  0  89   0   0]
 [  0   2  16   0]
 [  0   0   0  13]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       312
           1       0.96      1.00      0.98        89
           2       1.00      0.89      0.94        18
           3       1.00      1.00      1.00        13

    accuracy                           0.99       432
   macro avg       0.99      0.97      0.98       432
weighted avg       0.99      0.99      0.99       432



Potem ustawiono parametry na 2, gdzie wyniki okazały się być znacznie gorsze niż w poprzednich przykładach. Wartości błędów były wyższe od poprzednich przykładów, natomiast wynik uczenia wyniósł 78%.

In [24]:
display_errors(2, 2)

Confusion matrix for fold #1
[[480 204]
 [  0 180]]
Classification report for fold #1
              precision    recall  f1-score   support

           0       1.00      0.70      0.82       684
           1       0.47      1.00      0.64       180

    accuracy                           0.76       864
   macro avg       0.73      0.85      0.73       864
weighted avg       0.89      0.76      0.79       864

Mean absolute error for fold #1
0.2361111111111111
Mean squared error for fold #1
0.2361111111111111
TP rate for fold #1 (unacc, acc, good, vgood)
[0.70175439 1.        ]
TN rate for fold #1 (unacc, acc, good, vgood)
[1.         0.70175439]
FP rate for fold #1 (unacc, acc, good, vgood)
[0.         0.29824561]
FN rate for fold #1 (unacc, acc, good, vgood)
[0.29824561 0.        ]
Confusion matrix for fold #2
[[526   0   0   0]
 [204   0   0   0]
 [ 69   0   0   0]
 [ 65   0   0   0]]
Classification report for fold #2
              precision    recall  f1-score   support

           

/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
train(0.25, 2, 2)

[[242  61   0   0]
 [  0  96   0   0]
 [  0  16   0   0]
 [  0  17   0   0]]
              precision    recall  f1-score   support

           0       1.00      0.80      0.89       303
           1       0.51      1.00      0.67        96
           2       0.00      0.00      0.00        16
           3       0.00      0.00      0.00        17

    accuracy                           0.78       432
   macro avg       0.38      0.45      0.39       432
weighted avg       0.81      0.78      0.77       432



/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Następnie spróbowano zwiększyć podział danych treningowych i testowych na 50%. Wynik okazał się być niewiele lepszy od wyniku z poprzedniego ćwiczenia.

In [26]:
train(0.5, 5, 5)

[[573  44   0   2]
 [ 17 169   0   6]
 [  0  16   0  13]
 [  0   3   0  21]]
              precision    recall  f1-score   support

           0       0.97      0.93      0.95       619
           1       0.73      0.88      0.80       192
           2       0.00      0.00      0.00        29
           3       0.50      0.88      0.64        24

    accuracy                           0.88       864
   macro avg       0.55      0.67      0.60       864
weighted avg       0.87      0.88      0.87       864



/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Następnie spróbowano zwiększyć podział danych treningowych i testowych na 10%. Wynik okazał się być niezmieniony w stosunku do wyniku z poprzedniego ćwiczenia.

In [27]:
train(0.1, 5, 5)

[[100   9   0   1]
 [  4  44   0   0]
 [  0   9   0   3]
 [  0   0   0   3]]
              precision    recall  f1-score   support

           0       0.96      0.91      0.93       110
           1       0.71      0.92      0.80        48
           2       0.00      0.00      0.00        12
           3       0.43      1.00      0.60         3

    accuracy                           0.85       173
   macro avg       0.52      0.71      0.58       173
weighted avg       0.82      0.85      0.83       173



/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/patryk/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Wnioski

Liczba podziałów i głębokość drzewa decyzyjnego miały istotny wpływ na dokładność trenowania. Większe wartości obu parametrów powodowały wzrost dokładności, a największą dokładność drzewo osiągnęło przy maksymalnej liczbie podziałów i maksymalnej głębokości drzewa decyzyjnego. Ustalono również, że podział danych treningowych i testowych nie wpływał znacząco na skuteczność uczenia maszynowego. 